<a href="https://colab.research.google.com/github/prakher2pratyush/amazon_reviews/blob/main/amazon_fine_food_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Data

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Setup Connection
con = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/Data/amazon_fine_food_review.sqlite')

# Check Connection
def chk_conn(conn):
     try:
        conn.cursor()
        return True
     except Exception as ex:
        return False

print("Check if Connection is eastablished : " + str(chk_conn(con)))

# Check tables in db
tables = pd.read_sql_query("""
SELECT name 
FROM sqlite_master 
WHERE type='table'
""", con)

print('\nTables inside Database.')
print(tables)

# Add data
data = pd.read_sql_query("""
SELECT * 
FROM Reviews 
WHERE Score != 3
""", con)

print('\nShape of data.')
print(data.shape)
print('\nColumns in data.')
print(data.columns)
print('\n')
data.head()

Check if Connection is eastablished : True

Tables inside Database.
      name
0  Reviews

Shape of data.
(525814, 10)

Columns in data.
Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')




,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


###Fetch Data

In [ ]:
data.iloc[178139]['Text']

'Great product not too expensive easy to use. Ships quickly.<br />Easy to bake with. Please check out the fat to skinny book.'

##Update Data

In [ ]:
# Update Data to replace Score with Positive or Negative review.
def update_function(score):
  if score > 3:
    return 'Positive'
  return 'Negative'

positive_negative_data = data['Score'].map(update_function)
updated_data = data
updated_data['Score'] = positive_negative_data

updated_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,Negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


##Data Cleaning

In [ ]:
# Sort the data
sorted_data = updated_data.sort_values('ProductId')

# Remove Duplicates
filtered_data = sorted_data.drop_duplicates(subset = ['UserId', 'Time', 'Text'], keep = 'first')

print('\nShape of data.')
print(filtered_data.shape)
print('\nPercentage of data retained.')
print((filtered_data.size/sorted_data.size)*100)
print('\n')
print((filtered_data.count()/sorted_data.count())*100)

# Remove Irrelevant data
filtered_data = filtered_data[filtered_data['HelpfulnessNumerator']<=filtered_data['HelpfulnessDenominator']]

print('\nShape of data.')
print(filtered_data.shape)


Shape of data.
(364133, 10)

Percentage of data retained.
69.25129418387492


Id                        69.251294
ProductId                 69.251294
UserId                    69.251294
ProfileName               69.251294
HelpfulnessNumerator      69.251294
HelpfulnessDenominator    69.251294
Score                     69.251294
Time                      69.251294
Summary                   69.251294
Text                      69.251294
dtype: float64

Shape of data.
(364131, 10)


In [ ]:
# Check for valid values 
book_data = filtered_data.loc[filtered_data['Text'].str.contains('book')]
book_data.head()
book_data.shape
# My approach to remove non food reviews by going to https://amazon.com/dp/<product_id>
# and do a page scrapping to check the category. If category is non food related, will remove the entry.

(2333, 10)

## Text Preprocessing 
####(Read about Stemmimg and Lemmatization)

In [ ]:
# For Status Bar
from tqdm import tqdm
import re
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer

# We are removing the words from the stop words list: 'no', 'nor', 'not' and adding 'br'
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

preprocessed_reviews = []

for sentance in tqdm(filtered_data['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)                  # Remove URL
    sentance = BeautifulSoup(sentance, 'lxml').get_text()        # Remove Tags               
    sentance = decontracted(sentance)                            # Function to remove "'"
    sentance = re.sub("\S*\d\S*", "", sentance).strip()          # Remove Numeric/Alphanumeric values
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)               # Remove Special Characters
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)  # Remove Stopwords
    preprocessed_reviews.append(sentance.strip())                # Add list of words to form 2d Array

len(preprocessed_reviews)

100%|██████████| 364131/364131 [01:54<00:00, 3180.99it/s]


364131

##Bag of Words (BOW) and n-gram

In [ ]:
# BOW
count_vector = CountVectorizer()
bow_for_data = count_vector.fit_transform(preprocessed_reviews)
print(bow_for_data.shape)

# n-gram
count_vector = CountVectorizer(ngram_range = (1,2))
uni_and_bi_gram_for_data = count_vector.fit_transform(preprocessed_reviews)
print(uni_and_bi_gram_for_data.shape)

(364131, 116756)
(364131, 3923364)


## tf-idf

In [ ]:
# https://stackoverflow.com/questions/27697766/understanding-min-df-and-max-df-in-scikit-countvectorizer
tfidf_vector = TfidfVectorizer(ngram_range = (1,2))
tf_idf = tfidf_vector.fit_transform(preprocessed_reviews)
print(tf_idf.shape)

# ----------------------------------
# We use tf-idf on unprocessed data
# ----------------------------------
tf_idf_using_unprocessed_data = tfidf_vector.fit_transform(filtered_data['Text'].values)
print(tf_idf_using_unprocessed_data.shape)

(364131, 3923364)
(364131, 2910192)


##Word2Vec

In [ ]:
from gensim.models import Word2Vec

# In real life we can use https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
# w2v_model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
# Do note, its large file

list_of_sentance=[]
for sentance in tqdm(preprocessed_reviews):
    list_of_sentance.append(sentance.split())

w2v_model = Word2Vec(list_of_sentance, min_count = 5, size = 50, workers = 4)

100%|██████████| 364131/364131 [00:02<00:00, 127647.68it/s]


In [ ]:
w2v_model.wv.most_similar('tasty')

[('delicious', 0.8281922340393066),
 ('satisfying', 0.8199484348297119),
 ('tastey', 0.8006098866462708),
 ('yummy', 0.7882150411605835),
 ('filling', 0.7585986852645874),
 ('flavorful', 0.7511757016181946),
 ('nutritious', 0.6699881553649902),
 ('surprisingly', 0.661712646484375),
 ('hearty', 0.6576239466667175),
 ('good', 0.6425130367279053)]

## Avg. W2V and TF-IDF W2V

In [ ]:
print(preprocessed_reviews[:10])
print(list_of_sentance[:10])

['witty little book makes son laugh loud recite car driving along always sing refrain learned whales india drooping roses love new words book introduces silliness classic book willing bet son still able recite memory college', 'grew reading sendak books watching really rosie movie incorporates love son loves however miss hard cover version paperbacks seem kind flimsy takes two hands keep pages open', 'fun way children learn months year learn poems throughout school year like handmotions invent poem', 'great little book read aloud nice rhythm well good repetition little ones like lines chicken soup rice child gets go months year go wonderful places like bombay nile eating well know get eat kids maurice sendak version ice skating treat roses heads long time not even know came surprise came little witty book', 'book poetry months year goes month cute little poem go along love book really fun way learn months poems creative author purpose writing book give children fun way learn months chi

In [ ]:
# Avg. W2V

# Get valid words in a list.
w2v_words = list(w2v_model.wv.vocab)

avg_w2v = [];

for sentance in tqdm(list_of_sentance):
  sum_w2v = np.zeros(50)
  total_words = 0
  for word in sentance:
    if word in w2v_words:
      vec = w2v_model.wv[word]
      sum_w2v += vec              # w2v(w[1]) + w2v(w[2]) + ----- w2v(w[total_words])
      total_words += 1            # count total valid words after w2v in a sentance
    if total_words != 0:
      sum_w2v /= total_words      # 1/total_words*(w2v(w[1]) + w2v(w[2]) + ----- w2v(w[total_words]))
    avg_w2v.append(sum_w2v)

print(len(avg_w2v))
print(len(avg_w2v[0]))

100%|██████████| 364131/364131 [33:18<00:00, 182.23it/s]

14255129
50


In [ ]:
# TD-IDF W2V

tdidf_w2v = [];
tfidf_dictionary = dict(zip(tfidf_vector.get_feature_names(), list(tfidf_vector.idf_)))
tfidf_feature_names = tfidf_vector.get_feature_names()

for sentance in tqdm(list_of_sentance):
  sum_w2v = np.zeros(50)
  total_tfidf = 0
  for word in sentance:
    if word in w2v_words and word in tfidf_feature_names:
      vec = w2v_model.wv[word]
      sum_w2v += vec*(tfidf_dictionary[word]*(sentance.count(word)/len(sentance)))
      total_tfidf += (tfidf_dictionary[word]*(sentance.count(word)/len(sentance)))                             
    if total_tfidf != 0:
      sum_w2v /= total_tfidf                          
    tdidf_w2v.append(sum_w2v)

print(len(tdidf_w2v))
print(len(tdidf_w2v[0]))

Streaming output truncated to the last 5000 lines.
100%|██████████| 1/1 [00:00<00:00, 13.93it/s]

 60%|██████    | 33/55 [00:07<00:05,  3.92it/s]